# COGS 108 - Final Project (change this to your project's title)

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that student names will be included (but PIDs will be scraped from any groups who include their PIDs).

* [  ] YES - make available
* [  ] NO - keep private

# Overview

*Fill in your overview here*

# Names

- Ant Man
- Hulk
- Iron Man
- Thor
- Wasp

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

<a id='background'></a>

## Background & Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
## YOUR CODE HERE

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [3]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

There are many ethical concerns in the industry of music streaming, such as the income of musicians (especially independent artists).

Spotify's Code of Conduct is posted publicly [^3], where they claim that they comply with laws and are fully fair and in line with their interests. Additionally, they provide many links for contact if any individual suspects fraud or theft of Spotify assets. Spotify has a separate Privacy Policy, where they go into detail of the personal rights of users [^4]. 

There is potential for bias in the Spotify weekly charts, as popular artists are more likely to have a significant following, leading to an overrepresentation in the charts. Therefore, independent and/or lesser known artists do not have the same opportunities. Promotion from record labels who can afford to advertise their artists/songs would have the upper hand in this field. Additionally, we do not know the specifics of the algorithms used to generate Spotify's charts, which would potentially harbor bias in its model. 

Much of these are out of control, and since Spotify has made the Data Publicly available about the artists, and we are not directly working with individual user data, we assume that all practices are legel.

[^3]: https://s29.q4cdn.com/175625835/files/doc_downloads/gov-docs/Spotify-Code-of-Conduct-and-Ethics-(Updated-2020).pdf 

[^4]: https://www.spotify.com/us/legal/privacy-policy/

# Conclusion & Discussion

*Fill in your discussion information here*

# Team Contributions

*Specify who in your group worked on which parts of the project.*